In [6]:
# !pip install gymnasium
# !pip install "gymnasium[atari, accept-rom-license]"
# !apt-get install -y swig
# !pip install gymnasium[box2d]

In [7]:
# ===========================
# IMPORTS & GLOBAL SETUP
# ===========================
import os
import json
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from collections import deque
from collections import defaultdict
from datetime import datetime

import torch.nn.functional as F

SEED = 42

In [8]:
import warnings
import logging

# -----------------------
# WARNINGS UNTERDRÜCKEN
# -----------------------
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Gym / MoviePy Logging leiser machen
logging.getLogger("gymnasium").setLevel(logging.ERROR)
logging.getLogger("moviepy").setLevel(logging.ERROR)

# SDL / Pygame Headless Fix (verhindert XDG_RUNTIME_DIR Meldung)
os.environ["SDL_AUDIODRIVER"] = "dummy"
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [9]:
env_name = "LunarLander-v3"
env = gym.make(env_name)
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


In [10]:
class QNetwork(nn.Module):
    """
    Dynamisches MLP für DQN.
    Linear + ReLU nur zwischen Hidden-Layern
    """
    def __init__(self, state_dim, action_dim, layer_sizes=[64, 8, 64, 64]):
        super().__init__()
        torch.manual_seed(42)

        layers = []
        input_size = state_dim
        for i, size in enumerate(layer_sizes):
            layers.append(nn.Linear(input_size, size))
            # ReLU nur nach Hidden-Layern, nicht nach dem letzten Layer
            if i != len(layer_sizes) - 1:
                layers.append(nn.ReLU())
            input_size = size

        # Output Layer
        layers.append(nn.Linear(input_size, action_dim))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [11]:
class ReplayBuffer:
    """
    Speichert vergangene Erfahrungen:
    (state, action, reward, next_state, done)

    Warum?
    - bricht zeitliche Korrelationen
    - alte Erfahrungen können mehrfach gelernt werden
    """

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def add(self, experience):
        """
        Fügt eine Erfahrung hinzu.
        Wenn der Speicher voll ist → die älteste fliegt raus.
        """
        self.memory.append(experience)
        if len(self.memory) > self.capacity:
            self.memory.pop(0)

    def sample(self, batch_size):
        """
        Zieht zufällige Erfahrungen für ein Lern-Update
        """
        batch = random.sample(self.memory, batch_size)

        states      = torch.tensor(np.vstack([e[0] for e in batch]), dtype=torch.float32).to(self.device)
        actions     = torch.tensor(np.vstack([e[1] for e in batch]), dtype=torch.long).to(self.device)
        rewards     = torch.tensor(np.vstack([e[2] for e in batch]), dtype=torch.float32).to(self.device)
        next_states = torch.tensor(np.vstack([e[3] for e in batch]), dtype=torch.float32).to(self.device)
        dones       = torch.tensor(np.vstack([e[4] for e in batch]), dtype=torch.float32).to(self.device)

        return states, actions, rewards, next_states, dones


In [12]:
class DQNAgent:
    """
    DQN-Agent mit dynamischem QNetwork (beliebige Layer-Architektur)
    """

    def __init__(
        self,
        state_dim,
        action_dim,
        learning_rate,
        buffer_size,
        batch_size,
        gamma,
        tau,
        layer_info=None,  # <-- Neue Option für dynamische Architekturen
    ):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.batch_size = batch_size
        self.gamma = gamma          # Discount-Faktor
        self.tau = tau              # Soft-Update-Faktor bzw interpolation faktor
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.seed = SEED

        # Standard-Layer falls nichts angegeben
        if layer_info is None:
            layer_info = [64, 64, 64]

        # Online-Netzwerk
        self.q_network = QNetwork(state_dim, action_dim, layer_info).to(self.device)

        # Target-Netzwerk
        self.target_q_network = QNetwork(state_dim, action_dim, layer_info).to(self.device)
        self.target_q_network.load_state_dict(self.q_network.state_dict())  # initial synchronisieren

        # Optimizer
        self.optimizer = torch.optim.Adam(self.q_network.parameters(), lr=learning_rate)

        # Replay Buffer
        self.replay_buffer = ReplayBuffer(buffer_size)

        self.step_counter = 0

    def select_action(self, state, epsilon = 0.):
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)

        self.q_network.eval()
        with torch.no_grad():
            q_values = self.q_network(state)
        self.q_network.train()

        if random.random() > epsilon:
            return q_values.argmax(dim=1).item()
        else:
            return random.randrange(self.action_dim)

    def step(self, state, action, reward, next_state, done):
        self.replay_buffer.add((state, action, reward, next_state, done))
        self.step_counter += 1

        # Nicht bei jedem Schritt lernen (stabiler)
        if self.step_counter % 4 == 0:
            if len(self.replay_buffer.memory) >= self.batch_size:
                batch = self.replay_buffer.sample(self.batch_size)
                self.learn(batch)

    def learn(self, experiences):
        states, actions, rewards, next_states, dones = experiences

        with torch.no_grad():
            next_q_values = self.target_q_network(next_states).max(dim=1, keepdim=True)[0]
            q_targets = rewards + self.gamma * next_q_values * (1 - dones)

        q_expected = self.q_network(states).gather(1, actions)

        loss = F.mse_loss(q_expected, q_targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.soft_update()

    def soft_update(self):
        for target_param, local_param in zip(
            self.target_q_network.parameters(),
            self.q_network.parameters()
        ):
            target_param.data.copy_(
                self.tau * local_param.data +
                (1.0 - self.tau) * target_param.data
            )


In [13]:
def get_layer_info(agent_or_model):
    """
    Gibt nur die Größe der Linear-Layer zurück, z.B. [64, 64]
    """
    if hasattr(agent_or_model, "q_network"):
        model = agent_or_model.q_network
    else:
        model = agent_or_model

    layer_sizes = []
    for layer in model.modules():
        if isinstance(layer, nn.Linear) and layer.out_features != model.model[-1].out_features:
            # Letzter Linear-Layer (Output) ignorieren
            layer_sizes.append(layer.out_features)
    return layer_sizes

# -------------------------------
# load_model_bundle: rekonstruiert Linear+ReLU Modell
# -------------------------------
def save_model_bundle(model, save_dir, state_dim, action_dim, fitness=None, generation=None):
    os.makedirs(save_dir, exist_ok=True)
    # 1️⃣ Weights speichern
    torch.save(model.state_dict(), os.path.join(save_dir, "model.pth"))

    # 2️⃣ Meta speichern
    meta = {
        "state_dim": int(state_dim),
        "action_dim": int(action_dim),
        "layer_info": get_layer_info(model),
        "fitness": float(fitness) if fitness is not None else None,
        "generation": int(generation) if generation is not None else None,
        "seed": int(SEED) if SEED is not None else None,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    with open(os.path.join(save_dir, "meta.json"), "w") as f:
        json.dump(meta, f, indent=4)
    print(f"✓ Model saved to {save_dir}")

def load_model_bundle(model_dir, device="cpu"):
    # Meta laden
    with open(os.path.join(model_dir, "meta.json"), "r") as f:
        meta = json.load(f)
    state_dim = meta["state_dim"]
    action_dim = meta["action_dim"]
    layer_info = meta["layer_info"]

    # Modell rekonstruieren
    model = QNetwork(state_dim, action_dim, layer_info).to(device)

    # Gewichte laden (reihenfolgebasiert)
    state_dict_saved = torch.load(os.path.join(model_dir, "model.pth"), map_location=device)
    state_dict_model = model.state_dict()
    for k_model, k_saved in zip(state_dict_model.keys(), state_dict_saved.keys()):
        state_dict_model[k_model] = state_dict_saved[k_saved]
    model.load_state_dict(state_dict_model)

    return model, meta


def init_training_run(env_name, base_dir="training_history"):
    """
    Erstellt einen neuen Run-Ordner mit Timestamp
    """
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_dir = os.path.join(base_dir, f"{env_name}_{timestamp}")
    os.makedirs(run_dir, exist_ok=True)

    print(f"📁 Neuer Trainingslauf: {run_dir}")
    return run_dir

def init_training_log(run_dir):
    """
    Erstellt CSV-Logdatei für einen Trainingslauf
    """
    csv_path = os.path.join(run_dir, "training_log.csv")

    with open(csv_path, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow([
            "Environment",
            "Generation",
            "Model_Architecture",
            "Fitness",
            "Trained_Episodes",
        ])

    print(f"📊 Training-Log erstellt: {csv_path}")
    return csv_path
    
def append_generation_log(
    csv_path,
    env_name,
    generation,
    hidden_sizes,
    fitness,
    episodes_per_individual,
):
    with open(csv_path, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=";")

        writer.writerow([
            env_name,
            generation,
            str(hidden_sizes),
            f"{fitness:.2f}",
            generation * episodes_per_individual,
        ])


In [14]:
class PopulationMember:
    def __init__(self, agent):
        self.agent = agent
        self.total_reward = 0.0
        self.episode_rewards = []

def select_parents(sorted_population):
    return sorted_population[0], sorted_population[1]

def check_architecture_dominance(
    architecture_win_counter,
    threshold=6,
):
    for arch, wins in architecture_win_counter.items():
        if wins >= threshold:
            return list(arch)
    return None


In [15]:
# =========================
# 🚀 MUTATION
# =========================
def mutate_architecture(
    layer_info, 
    n_mutations=1, 
    min_neurons=8, 
    max_neurons=128
):
    """
    Mutiert ein Layer-Array n_mutations Mal.
    Optionen pro Mutation:
    1️⃣ Dupliziere oder lösche eine zufällige Schicht (Layer-Anzahl begrenzt auf min_layers/max_layers)
    2️⃣ Neuronen *2 oder /2 (begrenzt auf min_neurons/max_neurons)
    """
    new_layers = layer_info.copy()
    for _ in range(n_mutations):
        if len(new_layers) == 0:
            # Falls alle Layer gelöscht wurden, mindestens 1 Layer wieder hinzufügen
            new_layers.append(min_neurons)
            continue

        idx = random.randint(0, len(new_layers)-1)
        op = random.choice(["layer_op", "scale"])

        if op == "layer_op":
            # Duplizieren oder löschen, Layer-Anzahl beachten
            if len(new_layers) == 1:
                # Nur duplizieren möglich
                if len(new_layers) < max_layers:
                    new_layers.insert(idx, new_layers[idx])
            else:
                if random.random() < 0.5:
                    # duplizieren (nur wenn max_layers nicht überschritten)
                    if len(new_layers) < max_layers:
                        new_layers.insert(idx, new_layers[idx])
                else:
                    # löschen (nur wenn min_layers nicht unterschritten)
                    if len(new_layers) > min_layers:
                        new_layers.pop(idx)
        else:
            # Neuronen *2 oder /2
            factor = random.choice([0.5, 2])
            new_layers[idx] = max(min_neurons, min(max_neurons, int(new_layers[idx] * factor)))

    return new_layers


def create_initial_population(population_size, state_dim, action_dim, min_layers=1, max_layers=10,
                              min_neurons=8, max_neurons=128):
    """
    Initialisiert Population. Layer-Größe als Faktor von min_neurons.
    """
    population = []
    for _ in range(population_size):
        num_layers = random.randint(min_layers, max_layers)
        layers = [random.randint(1, max_neurons // min_neurons) * min_neurons for _ in range(num_layers)]
        agent = DQNAgent(
            state_dim=state_dim,
            action_dim=action_dim,
            learning_rate=learning_rate,
            buffer_size=buffer_size,
            batch_size=batch_size,
            gamma=gamma,
            tau=tau,
            layer_info=layers
        )
        population.append(PopulationMember(agent))
    return population

def create_child_from_parents(parent_agent, mutation_factor=5, epsilon=1, min_mutations=1):
    n_mutations = max(min_mutations, int(mutation_factor * epsilon))
    new_layers = mutate_architecture(get_layer_info(parent_agent), n_mutations=n_mutations)
    child_agent = DQNAgent(
        state_dim=state_size,
        action_dim=number_actions,
        learning_rate=learning_rate,
        buffer_size=buffer_size,
        batch_size=batch_size,
        gamma=gamma,
        tau=tau,
        layer_info=new_layers
    )
    return PopulationMember(child_agent)

def check_architecture_dominance(win_counter, threshold=6):
    for arch, count in win_counter.items():
        if count >= threshold:
            return list(arch)
    return None

In [16]:
def train_agent_for_episodes(
    env,
    agent,
    num_episodes,
    generation,
    max_steps_per_episode,
    epsilon_start,
    epsilon_end,
    epsilon_decay,
):
    """
    Trainiert einen Agenten für num_episodes Episoden.
    ε ist strikt lokal.
    Gibt Mean-Reward zurück.
    """
    epsilon = epsilon_start
    rewards = []

    for ep in range(num_episodes):
        reset_out = env.reset(seed=generation * 10_000 + ep)
        state = reset_out[0] if isinstance(reset_out, tuple) else reset_out
        episode_reward = 0.0

        for _ in range(max_steps_per_episode):
            action = agent.select_action(state, epsilon)

            step_out = env.step(action)
            if len(step_out) == 5:
                next_state, reward, terminated, truncated, _ = step_out
                done = terminated or truncated
            else:
                next_state, reward, done, _ = step_out

            agent.step(state, action, reward, next_state, done)
            state = next_state
            episode_reward += reward

            if done:
                break

        rewards.append(episode_reward)

        # ε nur lokal decayn
        epsilon = max(epsilon_end, epsilon * epsilon_decay)

    return float(np.mean(rewards))

In [17]:
def run_trained_model_and_record(
    model_dir,
    env_name="CartPole-v1",
    video_dir="videos",
    max_steps=500,
    device="cpu"
):

    os.makedirs(video_dir, exist_ok=True)

    # --------
    # Modell + Metadaten laden
    # --------
    model, meta = load_model_bundle(model_dir, device=device)
    model.eval()
    print(f"Lade Modell aus '{model_dir}' mit Layer-Info: {meta['layer_info']}")

    # --------
    # ENV vorbereiten + Video-Wrapper
    # --------
    env = gym.make(env_name, render_mode="rgb_array")
    env = RecordVideo(
        env,
        video_folder=video_dir,
        episode_trigger=lambda ep: True,  # jedes Episode aufnehmen
        name_prefix="episode"
    )

    reset_out = env.reset()
    state = reset_out[0] if isinstance(reset_out, tuple) else reset_out
    total_reward = 0

    # --------
    # Episode ausführen
    # --------
    for step in range(max_steps):
        state_t = torch.FloatTensor(state).unsqueeze(0).to(device)

        with torch.no_grad():
            q_values = model(state_t)
            action = torch.argmax(q_values, dim=-1).item()

        step_out = env.step(action)
        if len(step_out) == 5:
            state, reward, terminated, truncated, _ = step_out
            done = terminated or truncated
        else:
            state, reward, done, _ = step_out

        total_reward += reward
        if done:
            break

    env.close()

    print(f"Episode Länge: {step+1}")
    print(f"Return: {total_reward}")
    print(f"Video gespeichert in: {video_dir}")


In [18]:
learning_rate = 5e-4
buffer_size = int(1e5)
batch_size = 50
gamma = 0.99
tau = 1e-3

# -----------------------------
# Evolutionäre Trainingsparameter
# -----------------------------

population_size = 10
mutation_factor = 5
thresholddom = 5
min_layers=1 
max_layers=5

max_steps_per_episode = 1000
goal = 200.0

epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 0.995
scores_window = deque(maxlen=100)

population = create_initial_population(population_size, state_dim=state_size, action_dim=number_actions)
architecture_win_counter = defaultdict(int)
dominant_architecture = None

In [19]:
# -----------------------------
# Ordner + Log initialisieren
# -----------------------------
run_dir = init_training_run(env_name)
log_csv = init_training_log(run_dir)

# -----------------------------
# Trainingsloop
# -----------------------------
generation = 0
epsilon = epsilon_start

# =========================
# 🔹 Population trainieren
# =========================
while True:
    for member in population:
        mean_score = train_agent_for_episodes(
            env=env,
            agent=member.agent,
            num_episodes=batch_size,
            generation=generation,
            max_steps_per_episode=max_steps_per_episode,
            epsilon_start=epsilon,
            epsilon_end=epsilon_end,
            epsilon_decay=epsilon_decay,
        )
        member.total_reward = mean_score
        print(f"training von {get_layer_info(member.agent)} fertig")
        
    population.sort(key=lambda m: m.total_reward, reverse=True)
    print(f"\n🏅 Population Ranking – Generation {generation}")
    for rank, member in enumerate(population, start=1):
        print(
            f"{rank:2d}. Reward: {member.total_reward:8.2f} | "
            f"Layers: {get_layer_info(member.agent)}"
        )
        append_generation_log(
            csv_path=log_csv,
            env_name=env_name,
            generation=generation,
            hidden_sizes=get_layer_info(member.agent),
            fitness=member.total_reward,
            episodes_per_individual=batch_size
        )
    
    if population[0].total_reward >= goal:
        # 🔹 Nur das beste Modell speichern
        save_dir = os.path.join(run_dir, f"checkpoint_generation_{generation}")
        save_model_bundle(
            model=population[0].agent.q_network,
            save_dir=save_dir,
            state_dim=state_size,
            action_dim=number_actions,
            fitness=population[0].total_reward,
            generation=generation
        )
        print(f"\n🎯 Goal erreicht mit {get_layer_info(population[0].agent)} in Generation {generation}")
        break
    
    # Dominanz prüfen und +
    architecture_win_counter[tuple(get_layer_info(population[0].agent))] += 1
    dominant_architecture = check_architecture_dominance(architecture_win_counter, thresholddom)
    
    # 🔹 Neue Population erzeugen
    new_population = []
    if dominant_architecture:
        new_population = [population[0]]  # nur dominant weitertrainieren
    else:
        new_population = [population[0], population[1]]
        remaining_slots = population_size - 2
        n_children_p1 = int(remaining_slots * 0.5)
        n_children_p2 = remaining_slots - n_children_p1

        print(f"Kinder haben {max(1, int(mutation_factor * epsilon))} mutationen")
    
        for _ in range(n_children_p1):
            child = create_child_from_parents(population[0].agent, mutation_factor, epsilon)
            new_population.append(child)
        for _ in range(n_children_p2):
            child = create_child_from_parents(population[1].agent, mutation_factor, epsilon)
            new_population.append(child)
    
    population = new_population
    
    # 🔹 Epsilon decay
    epsilon = max(epsilon_end, epsilon * (epsilon_decay ** batch_size))
    generation += 1

📁 Neuer Trainingslauf: training_history/LunarLander-v3_2026-01-10_12-12-52
📊 Training-Log erstellt: training_history/LunarLander-v3_2026-01-10_12-12-52/training_log.csv
training von [96, 48, 128, 40, 72, 112] fertig
training von [32, 24, 96, 40, 112] fertig
training von [64, 16, 72, 104] fertig
training von [16, 24, 120, 48, 72, 16, 104, 96, 56, 112] fertig
training von [48, 8, 120] fertig
training von [72, 8, 96, 112, 16, 40] fertig
training von [128, 56, 16] fertig
training von [16, 112] fertig
training von [120, 120, 112, 80, 112, 8, 128, 72, 96] fertig
training von [112, 48, 56, 88, 56, 104, 120] fertig

🏅 Population Ranking – Generation 0
 1. Reward:  -173.44 | Layers: [128, 56, 16]
 2. Reward:  -197.89 | Layers: [96, 48, 128, 40, 72, 112]
 3. Reward:  -207.69 | Layers: [32, 24, 96, 40, 112]
 4. Reward:  -208.15 | Layers: [48, 8, 120]
 5. Reward:  -208.42 | Layers: [120, 120, 112, 80, 112, 8, 128, 72, 96]
 6. Reward:  -210.00 | Layers: [64, 16, 72, 104]
 7. Reward:  -212.58 | Laye

In [21]:
# Angenommen, das Modell wurde gespeichert in:
model_dir = "training_history/LunarLander-v3_2026-01-10_12-12-52/checkpoint_generation_16"

# Video ausführen und speichern
run_trained_model_and_record(
    model_dir=model_dir,
    env_name=env_name,                 # aus deinem Setup
    video_dir="videos",                # Ordner für Video
    max_steps=max_steps_per_episode,   # aus deinem Setup
    device="cpu"                       # falls GPU nicht verfügbar
)


Lade Modell aus 'training_history/LunarLander-v3_2026-01-10_12-12-52/checkpoint_generation_16' mit Layer-Info: [96, 128, 40, 18]


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"


Episode Länge: 227
Return: 295.5873778021281
Video gespeichert in: videos


In [ ]:
# !rm -rf checkpoints/
# !rm -rf training_history/
# !rm -rf videos/